In [ ]:
# !rm -rf ~/.cache/huggingface/datasets
# !pip install -U datasets
!pip install sentence_transformers

In [ ]:
# import packages
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('./data_for_paper/train_en_report.csv', index_col=False, encoding='utf-8')
train, valid = train_test_split(df, test_size=0.1)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
train

In [ ]:
# train= train.rename(columns={"input": "question", "output": "answer"})
# valid =valid.rename(columns={"input": "question", "output": "answer"})
# test = test.rename(columns={"input": "question", "output": "answer"})

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
import tiktoken
from collections import Counter
import math
#from kiwipiepy import Kiwi
import os
import pandas as pd
import multiprocessing
import numpy as np
from collections import Counter
from sklearn.metrics import mutual_info_score
import math
from kiwipiepy import Kiwi
import os
import pandas as pd
import multiprocessing
import time
from sentence_transformers import SentenceTransformer, util
import numpy as np
from collections import Counter
import torch
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
os.environ["TOKENIZERS_PARALLELISM"] = "false"



# 1
q_list = train['question'].tolist()
a_list = train['answer'].tolist()

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

q_ner_results = ner(q_list)
a_ner_results = ner(a_list)


num_q_ne_list = []
num_a_ne_list = []

for i in range(len(q_ner_results)):
    num_q_ne_list.append(len(q_ner_results[i]))
    num_a_ne_list.append(len(a_ner_results[i]))

train['num_q_ne'] = num_q_ne_list
train['num_a_ne'] = num_a_ne_list

# 2
q_ne_list = []
a_ne_list = []

for i in range(len(q_ner_results)):
    tmp = []
    for j in range(len(q_ner_results[i])):
        tmp.append(q_ner_results[i][j]['word'])
    q_ne_list.append(list(set(tmp)))

for i in range(len(a_ner_results)):
    tmp = []
    for j in range(len(a_ner_results[i])):
        tmp.append(a_ner_results[i][j]['word'])
    a_ne_list.append(list(set(tmp)))
        

ne_unique_diff = []
for i in range(len(q_ner_results)):
    ne_unique_diff.append(len(set(a_ne_list[i]) - set(q_ne_list[i])))
train['ne_unique_diff'] = ne_unique_diff

# 3
q_list = train['question'].tolist()
a_list = train['answer'].tolist()
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

q_embs = model.encode(q_list)
a_embs = model.encode(a_list)

sim_list = []
for i in range(len(q_embs)):
    sim = 1 - cosine(q_embs[i], a_embs[i])
    sim_list.append(sim)

train['st_sim'] = sim_list

# 4
# Load BERT model
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertModel.from_pretrained('google-bert/bert-base-uncased')

# Function to calculate BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(1).detach().numpy()

# Function to calculate semantic similarity
def semantic_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    return 1 - cosine(embedding1.flatten(), embedding2.flatten())

q_list = train['question'].tolist()
a_list = train['answer'].tolist()


sim_list = []
for i in range(len(q_list)):
    sim = semantic_similarity(q_list[i], a_list[i])
    sim_list.append(sim)

train['bert_sim'] = sim_list

# 5
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertModel.from_pretrained('google-bert/bert-base-uncased')

def get_embedding_variance(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.detach().numpy()
    variances = np.var(embeddings, axis=1)
    return np.mean(variances)

info_ratio_list = []
for i in range(len(q_list)):
    question_variance = get_embedding_variance(q_list[i])
    answer_variance = get_embedding_variance(a_list[i])
    information_content_ratio = answer_variance / question_variance
    info_ratio_list.append(information_content_ratio)

train['bert_info_ratio'] = info_ratio_list


tfidf_vectorizer = TfidfVectorizer()
combined_texts = q_list + a_list
tfidf_vectorizer.fit(combined_texts)

info_ratio_list = []
for i in range(len(q_list)):
    tfidf_question = tfidf_vectorizer.transform([q_list[i]]).toarray()
    tfidf_answer = tfidf_vectorizer.transform([a_list[i]]).toarray()
    # Measure information content
    information_content_ratio = np.sum(tfidf_answer) / np.sum(tfidf_question)
    info_ratio_list.append(information_content_ratio)
    
train['tf-idf_info_ratio'] = info_ratio_list


tokenizer_ = tiktoken.get_encoding("cl100k_base")
allowed_special = tokenizer_.special_tokens_set

def process_chunk(chunk):
    chunk['len_q_character'] = chunk['question'].apply(len)
    chunk['len_a_character'] = chunk['answer'].apply(len)

    chunk['len_q_token'] = chunk['question'].apply(lambda x: len(tokenizer_.encode(x, allowed_special=allowed_special)))
    chunk['len_a_token'] = chunk['answer'].apply(lambda x: len(tokenizer_.encode(x, allowed_special=allowed_special)))

    return chunk


chunk_size = math.ceil(len(train) / 80)  # Divide data into chunks for 80 CPU cores
chunks = [train[i:i + chunk_size] for i in range(0, len(train), chunk_size)]

with multiprocessing.Pool(processes=80) as pool:
    processed_chunks = pool.map(process_chunk, chunks)

train = pd.concat(processed_chunks)

def calculate_entropy(sentence):
    tokens = tokenizer_.encode(sentence, allowed_special=allowed_special)
    #tokens = [tmp[i].form for i in range(len(tmp))]
    token_freq = Counter(tokens)
    total_tokens = len(tokens)
    entropy = -sum((freq / total_tokens) * math.log2(freq / total_tokens) for freq in token_freq.values())
    return entropy

def process_chunk(chunk):
    chunk['q_entropy'] = chunk['question'].apply(calculate_entropy)
    chunk['a_entropy'] = chunk['answer'].apply(calculate_entropy)
    return chunk


chunk_size = math.ceil(len(train) / 80)  # Divide data into chunks for 80 CPU cores
chunks = [train[i:i + chunk_size] for i in range(0, len(train), chunk_size)]

with multiprocessing.Pool(processes=80) as pool:
    processed_chunks = pool.map(process_chunk, chunks)

train = pd.concat(processed_chunks)


In [ ]:
train.to_csv('./data_for_paepr/en_report_train_features.csv', encoding='utf-8',index=False)

In [ ]:
### valid

In [ ]:
train = valid

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
import tiktoken
from collections import Counter
import math
#from kiwipiepy import Kiwi
import os
import pandas as pd
import multiprocessing
import numpy as np
from collections import Counter
from sklearn.metrics import mutual_info_score
import math
from kiwipiepy import Kiwi
import os
import pandas as pd
import multiprocessing
import time
from sentence_transformers import SentenceTransformer, util
import numpy as np
from collections import Counter
import torch
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
os.environ["TOKENIZERS_PARALLELISM"] = "false"



# 1
q_list = train['question'].tolist()
a_list = train['answer'].tolist()

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

q_ner_results = ner(q_list)
a_ner_results = ner(a_list)


num_q_ne_list = []
num_a_ne_list = []

for i in range(len(q_ner_results)):
    num_q_ne_list.append(len(q_ner_results[i]))
    num_a_ne_list.append(len(a_ner_results[i]))

train['num_q_ne'] = num_q_ne_list
train['num_a_ne'] = num_a_ne_list

# 2
q_ne_list = []
a_ne_list = []

for i in range(len(q_ner_results)):
    tmp = []
    for j in range(len(q_ner_results[i])):
        tmp.append(q_ner_results[i][j]['word'])
    q_ne_list.append(list(set(tmp)))

for i in range(len(a_ner_results)):
    tmp = []
    for j in range(len(a_ner_results[i])):
        tmp.append(a_ner_results[i][j]['word'])
    a_ne_list.append(list(set(tmp)))
        

ne_unique_diff = []
for i in range(len(q_ner_results)):
    ne_unique_diff.append(len(set(a_ne_list[i]) - set(q_ne_list[i])))
train['ne_unique_diff'] = ne_unique_diff

# 3
q_list = train['question'].tolist()
a_list = train['answer'].tolist()
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

q_embs = model.encode(q_list)
a_embs = model.encode(a_list)

sim_list = []
for i in range(len(q_embs)):
    sim = 1 - cosine(q_embs[i], a_embs[i])
    sim_list.append(sim)

train['st_sim'] = sim_list

# 4
# Load BERT model
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertModel.from_pretrained('google-bert/bert-base-uncased')

# Function to calculate BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(1).detach().numpy()

# Function to calculate semantic similarity
def semantic_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    return 1 - cosine(embedding1.flatten(), embedding2.flatten())

q_list = train['question'].tolist()
a_list = train['answer'].tolist()


sim_list = []
for i in range(len(q_list)):
    sim = semantic_similarity(q_list[i], a_list[i])
    sim_list.append(sim)

train['bert_sim'] = sim_list

# 5
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertModel.from_pretrained('google-bert/bert-base-uncased')

def get_embedding_variance(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.detach().numpy()
    variances = np.var(embeddings, axis=1)
    return np.mean(variances)

info_ratio_list = []
for i in range(len(q_list)):
    question_variance = get_embedding_variance(q_list[i])
    answer_variance = get_embedding_variance(a_list[i])
    information_content_ratio = answer_variance / question_variance
    info_ratio_list.append(information_content_ratio)

train['bert_info_ratio'] = info_ratio_list

tfidf_vectorizer = TfidfVectorizer()
combined_texts = q_list + a_list
tfidf_vectorizer.fit(combined_texts)

info_ratio_list = []
for i in range(len(q_list)):
    tfidf_question = tfidf_vectorizer.transform([q_list[i]]).toarray()
    tfidf_answer = tfidf_vectorizer.transform([a_list[i]]).toarray()
    # Measure information content
    information_content_ratio = np.sum(tfidf_answer) / np.sum(tfidf_question)
    info_ratio_list.append(information_content_ratio)
    
train['tf-idf_info_ratio'] = info_ratio_list


# 7
tokenizer_ = tiktoken.get_encoding("cl100k_base")
allowed_special = tokenizer_.special_tokens_set

def process_chunk(chunk):
    chunk['len_q_character'] = chunk['question'].apply(len)
    chunk['len_a_character'] = chunk['answer'].apply(len)

    chunk['len_q_token'] = chunk['question'].apply(lambda x: len(tokenizer_.encode(x, allowed_special=allowed_special)))
    chunk['len_a_token'] = chunk['answer'].apply(lambda x: len(tokenizer_.encode(x, allowed_special=allowed_special)))

    return chunk


chunk_size = math.ceil(len(train) / 80)  # Divide data into chunks for 80 CPU cores
chunks = [train[i:i + chunk_size] for i in range(0, len(train), chunk_size)]

with multiprocessing.Pool(processes=80) as pool:
    processed_chunks = pool.map(process_chunk, chunks)

train = pd.concat(processed_chunks)

# 8
def calculate_entropy(sentence):
    tokens = tokenizer_.encode(sentence, allowed_special=allowed_special)
    #tokens = [tmp[i].form for i in range(len(tmp))]
    token_freq = Counter(tokens)
    total_tokens = len(tokens)
    entropy = -sum((freq / total_tokens) * math.log2(freq / total_tokens) for freq in token_freq.values())
    return entropy

def process_chunk(chunk):
    chunk['q_entropy'] = chunk['question'].apply(calculate_entropy)
    chunk['a_entropy'] = chunk['answer'].apply(calculate_entropy)
    return chunk


chunk_size = math.ceil(len(train) / 80)  # Divide data into chunks for 80 CPU cores
chunks = [train[i:i + chunk_size] for i in range(0, len(train), chunk_size)]

with multiprocessing.Pool(processes=80) as pool:
    processed_chunks = pool.map(process_chunk, chunks)

train = pd.concat(processed_chunks)


train.to_csv('./data_for_paepr/valid.csv', encoding='utf-8',index=False)